<a href="https://colab.research.google.com/github/achmadgani/devops_training/blob/main/colletothricum_growth_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INTRODUCTION TO PYTHON

# Introduction

### Colletothricum Clades Implementation

In [ ]:
print("hello world")

hello world


# New Section

Code for Colletothricum Clades implementation python Program

In [20]:
import pandas as pd
import numpy as np

class ColletotrichumGrowthCalculator:
    #constants
    HS = 1
    r = 0.05
    p = 6
    i = 28

    def __init__(self, csv_path,HS=1, r=0.05, p=6, i=28, RcOPT=0.1):
        self.csv_path = csv_path
        self.HS = HS
        self.r = r
        self.p = p
        self.i = i
        self.RcOPT = RcOPT
        self.df = pd.read_csv(csv_path)
        self.rename_columns()

    def rename_columns(self):
        # Rename columns by removing any parentheses and the content within them
        self.df.columns = [col.split('(')[0].strip() for col in self.df.columns]

    def primary_inoculum(self):
        # Assuming the DataFrame is already loaded and columns are renamed

        # f(WD)
        self.df['f_WD'] = np.where(self.df['wetness_duration'] >= 3,
                                    np.exp(-5.947 * np.exp(-0.067 * self.df['wetness_duration'])), 0)

        # Teq
        self.df['Teq'] = np.where((self.df['temperature'] > 40) | (self.df['temperature'] < 5),
                                   0, (self.df['temperature'] - 5) / (40 - 5))

        # f(T)
        self.df['f_T'] = (4.975 * self.df['Teq']**1.354 * (1 - self.df['Teq']))**9.409

        # f1(WD)
        self.df['f1_WD'] = np.where(self.df['wetness_duration'] >= 3,
                                     0.398449 * np.exp(-0.067 * self.df['wetness_duration']) -
                                     5.947 * np.exp(-0.067 * self.df['wetness_duration']), 0)

        # SPOR'
        self.df['SPOR_prime'] = 0.00004 * self.df['f_T'] * self.df['f1_WD']

        # Initialize CON column with zeros
        self.df['CON'] = 0

        # Loop through the DataFrame rows to calculate CON values
        for i in range(1, len(self.df)):
            self.df.loc[i, 'CON'] = self.df.loc[i - 1, 'CON'] + self.df.loc[i, 'SPOR_prime']

    def conidia_dispersal(self):
        # DISR_f1(P)
        self.df['DISR_f1_P'] = np.where(self.df['precipitation'] >= 0.3,
                                         np.exp(-2.5 * np.exp(-0.15 * self.df['precipitation'])), 0)

        # DISR_f2(P)
        self.df['DISR_f2_P'] = np.where(self.df['precipitation'] > 15,
                                         1 / (1 + 966.9 * np.exp(-0.133 * self.df['precipitation'])), 0)

        # DISR
        self.df['DISR'] = np.where(self.df['precipitation'] >= 0.3,
                                    self.df['DISR_f1_P'] * (1 - self.df['DISR_f2_P']), 0)


    def conidia_infection(self):
        # Calculate RcWD
        self.df['RcWD'] = 1 - np.exp(-0.1 * self.df['wetness_duration'])

        # Calculate RcWD'
        self.df['RcWD_prime'] = 0.1 * np.exp(-0.1 * self.df['wetness_duration'])

        # Calculate Teq(Rc)
        self.df['Teq_Rc'] = np.where(
            (self.df['temperature'] > 33) | (self.df['temperature'] < 5),
            0,
            (self.df['temperature'] - 5) / (33 - 5)
        )

        # Calculate RcT
        self.df['RcT'] = (5.908 * self.df['Teq_Rc'] ** 1.689 * (1 - self.df['Teq_Rc'])) ** 5.523

        # Calculate Rc using the constant RcOPT
        self.df['Rc'] = self.RcOPT * self.df['RcT'] * self.df['RcWD_prime']


    def incubation_latency_period(self):
        # Constants for the formulas
        Tmin = 5
        Topt = 24.2
        Tmax = 30.5
        LPmin = 140.4  # Added LPmin constant
        IPmin = 118.3  # Incubation period minimum constant

        # Calculations for incubation (I)
        self.df['Ta'] = self.df['temperature'] - Tmin
        self.df['Tb'] = Topt - Tmin
        self.df['Tc'] = Tmax - self.df['temperature']
        self.df['Td'] = Tmax - Topt
        self.df['Tc_Td'] = self.df['Tc'] / self.df['Td']
        self.df['Te'] = (Tmax - Topt) / (Topt - Tmin)
        self.df['Ta_Tb'] = self.df['Ta'] / self.df['Tb']
        self.df['Tc_Td_Te'] = self.df['Tc_Td'] ** self.df['Te']
        self.df['f_T_I'] = self.df['Ta_Tb'] * self.df['Tc_Td_Te']

        # Constants for the latency period (L)
        Taa_min = 14.5  # Assuming these are constants provided for latency period
        Tbb_min = 26.8
        Tcc_max = 23.5
        Tdd_max = 11.2

        # Calculations for latency period (L)
        self.df['Taa'] = self.df['temperature'] - Taa_min
        self.df['Tbb'] = Tbb_min - Taa_min
        self.df['Tcc'] = Tcc_max - self.df['temperature']
        self.df['Tdd'] = Tcc_max - Tbb_min
        self.df['Tcc_Tdd'] = self.df['Tcc'] / self.df['Tdd']
        self.df['Tee'] = (Tcc_max - Tbb_min) / (Tbb_min - Taa_min)
        self.df['Taa_Tbb'] = self.df['Taa'] / self.df['Tbb']
        self.df['Tcc_Tdd_Tee'] = self.df['Tcc_Tdd'] ** self.df['Tee']
        self.df['f_T_L'] = self.df['Taa_Tbb'] * self.df['Tcc_Tdd_Tee']

        # pi and pl calculation
        self.df['pi'] = self.df['f_T_I'] / IPmin
        self.df['pl'] = self.df['f_T_L'] / LPmin

    # def combine_results(self):
    #     # This combines all the results into a single DataFrame
    #     # You could select only the columns with the results
    #     relevant_columns = ['conidia_dispersal_result', 'conidia_infection_result', 'incubation_latency_period_result']
    #     # Add more columns from self.df as needed
    #     self.mathematical_result = self.df[relevant_columns].copy()

    # def calculate_growth(self):
    #     # Add the calculation logic here
    #     self.df['growth'] = self.df.apply(lambda row: self.formula(row), axis=1)

    # def formula(self, row):

    def save_results(self, output_path):
        self.df.to_csv(output_path, index=False)

    def display_dataframe(self):
    # Use the built-in display function to nicely format the output in Jupyter/Colab
        display(self.df)

# Usage:
# Initialize the calculator with the path to your CSV file
calculator = ColletotrichumGrowthCalculator('/content/mulberry - tsukuba data.csv')
calculator.primary_inoculum()
calculator.conidia_dispersal()
calculator.conidia_infection()
calculator.incubation_latency_period()

# Combine all the results into the mathematical_result DataFrame
# calculator.combine_results()

# Now you can access the combined results with:
# mathematical_result = calculator.mathematical_result

calculator.display_dataframe()
# calculator.save_results('mathematical_result.csv')


,datetime,temperature,precipitation,humidity,wetness_duration,f_WD,Teq,f_T,f1_WD,SPOR_prime,...,Tbb,Tcc,Tdd,Tcc_Tdd,Tee,Taa_Tbb,Tcc_Tdd_Tee,f_T_L,pi,pl
0,1993/7/1 1:00,16.5,0.0,81,0,0.000000,0.328571,0.058930,0.000000,0.000000,...,12.3,7.0,-3.3,-2.121212,-0.268293,0.162602,NaN,NaN,0.006580,NaN
1,1993/7/1 2:00,16.9,0.0,79,0,0.000000,0.340000,0.077510,0.000000,0.000000,...,12.3,6.6,-3.3,-2.000000,-0.268293,0.195122,NaN,NaN,0.006744,NaN
2,1993/7/1 3:00,16.6,0.0,80,0,0.000000,0.331429,0.063214,0.000000,0.000000,...,12.3,6.9,-3.3,-2.090909,-0.268293,0.170732,NaN,NaN,0.006621,NaN
3,1993/7/1 4:00,15.1,0.0,87,0,0.000000,0.288571,0.019434,0.000000,0.000000,...,12.3,8.4,-3.3,-2.545455,-0.268293,0.048780,NaN,NaN,0.005962,NaN
4,1993/7/1 5:00,15.8,0.0,85,0,0.000000,0.308571,0.034899,0.000000,0.000000,...,12.3,7.7,-3.3,-2.333333,-0.268293,0.105691,NaN,NaN,0.006279,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,1993/11/29 20:00,20.2,0.0,92,8,0.030823,0.434286,0.410819,-3.246368,-0.000053,...,12.3,3.3,-3.3,-1.000000,-0.268293,0.463415,NaN,NaN,0.007863,NaN
3644,1993/11/29 21:00,20.4,0.0,91,8,0.030823,0.440000,0.441053,-3.246368,-0.000057,...,12.3,3.1,-3.3,-0.939394,-0.268293,0.479675,NaN,NaN,0.007916,NaN
3645,1993/11/29 22:00,20.7,0.0,92,8,0.030823,0.448571,0.487762,-3.246368,-0.000063,...,12.3,2.8,-3.3,-0.848485,-0.268293,0.504065,NaN,NaN,0.007991,NaN
3646,1993/11/29 23:00,20.7,0.0,92,8,0.030823,0.448571,0.487762,-3.246368,-0.000063,...,12.3,2.8,-3.3,-0.848485,-0.268293,0.504065,NaN,NaN,0.007991,NaN
